In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [3]:
sources = {
    "litoral": {
        "violencia_narco": "https://www.ellitoral.com/tag/violencia-narco",
        "narcotrafico": "https://www.ellitoral.com/tag/narcotrafico",
    },
    "aire": {
        "droga": "https://www.airedesantafe.com.ar/droga-a848/",
        "narcotrafico": "https://www.airedesantafe.com.ar/narcotrafico-a534",
        "cocaina": "https://www.airedesantafe.com.ar/cocaina-a1378",
        "drogas": "https://www.airedesantafe.com.ar/drogas-a4787",
    },
    "rosario12": {
        "general": "https://www.pagina12.com.ar/suplementos/rosario12/{date}",
    },
}

In [4]:
def scrape_links_and_titles_litoral(url):
    # Send a GET request to the webpage
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the webpage content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the div with the specific class
        div = soup.find('div', class_='styles_temas-container__2moeC')

        if div:
            # Prepare a list to store the data
            data = []

            # Find all <a> tags within the div
            links = div.find_all('a', href=True)

            for link in links:
                href = url + link['href']  # Prepend the prefix to the href attribute

                # Find the <h1> inside the <a> tag (if it exists)
                h1 = link.find('h1')
                title = h1.get_text(strip=True) if h1 else None  # Extract text or use None if not found

                # Append the link and title to the data list
                data.append({'link': href, 'title': title})

            # Create a DataFrame from the data list
            df = pd.DataFrame(data)

            return df
        else:
            print("No div with the specified class was found.")
            return pd.DataFrame()  # Return an empty DataFrame
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame



NameError: name 'scrape_links_and_titles' is not defined

In [29]:
def scrape_with_multiple_clicks_litoral(sources):
    
    # Path to your Brave executable
    brave_path = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"

    # Set up Chrome options
    options = Options()
    options.binary_location = brave_path

    # Create a WebDriver instance
    driver = webdriver.Chrome(options=options)  

    # List to store all scraped data
    data = []

    # Extract URLs and tags
    urls = list(sources["litoral"].values())
    tags = list(sources["litoral"].keys())

    try:
        # Iterate through each URL and tag
        for url, tag in zip(urls, tags):
            try:
                # Load the webpage
                driver.get(url)

                # Wait for the page to load initially
                time.sleep(5)

                try:
                    cancel_button = driver.find_element(By.ID, "onesignal-slidedown-cancel-button")
                    cancel_button.click()
                    time.sleep(2)  # Wait some time after clicking the cancel button
                except NoSuchElementException:
                    print(f"The 'onesignal-slidedown-cancel-button' button not found on {url}.")
                    pass                

                # Click the button until it no longer exists
                while True:
                    try:                        
                        # Find and click the button
                        button = driver.find_element(By.CLASS_NAME, "styles_btn__Y8WwT")
                        driver.execute_script("arguments[0].scrollIntoView();", button)
                        button.click()
                        time.sleep(5)  # Wait for new content to load
                    # Scroll down the page
                        actions = ActionChains(driver)
                        actions.send_keys(Keys.END).perform()  # Scroll to the bottom of the page
                        time.sleep(2)  # Wait for the page to fully load after scrolling
                        try:
                            cancel_button = driver.find_element(By.ID, "onesignal-slidedown-cancel-button")
                            cancel_button.click()
                            time.sleep(2)  # Wait some time after clicking the cancel button
                        except NoSuchElementException:
                            print(f"The 'onesignal-slidedown-cancel-button' button not found on {url}.")
                            pass                
                            
                    except NoSuchElementException as e:
                        print(f"NoSuchElementException: Button not found on {url}. Error: {e}")
                        break
                    except ElementNotInteractableException as e:
                        print(f"ElementNotInteractableException: Button found but not interactable on {url}. Error: {e}")
                        break
                    except ElementClickInterceptedException as e:
                        print(f"ElementClickInterceptedException: Button click intercepted on {url}. Error: {e}")
                        break

                # Extract the final page source
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Scrape the content using BeautifulSoup
                div = soup.find('div', class_='styles_temas-container__2moeC')

                if div:
                    # Find all <a> tags within the div
                    links = div.find_all('a', href=True)

                    for link in links:
                        href = url + link['href']  # Prepend the URL prefix to the href attribute

                        # Find the <h1> inside the <a> tag (if it exists)
                        h1 = link.find('h1')
                        title = h1.get_text(strip=True) if h1 else None  # Extract text or use None if not found

                        # Append the link, title, and additional info to the data list
                        data.append({'Link': href, 'Title': title, 'Tag': tag, 'Media': 'ellitoral'})

                else:
                    print(f"No div with the specified class found on {url}.")

            except Exception as e:
                print(f"Error processing {url}: {e}")

    finally:
        # Close the WebDriver
        driver.quit()

    # Create a DataFrame from the data list
    return pd.DataFrame(data)

In [30]:
df_litoral = scrape_with_multiple_clicks_litoral(sources)

The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
The 'onesignal-slidedown-cancel-butto

In [36]:
df_litoral.to_excel("data/df_litoral_historic.xlsx", index=False)

In [3]:
df_litoral = pd.read_excel("data/df_litoral_historic.xlsx")
df_litoral

,Link,Title,Tag,Media
0,https://www.ellitoral.com/tag/violencia-narco/...,Asesinaron a tiros a un alcalde y otras tres p...,violencia_narco,ellitoral
1,https://www.ellitoral.com/tag/violencia-narco/...,A la cárcel: narcos barriales deberán pasar un...,violencia_narco,ellitoral
2,https://www.ellitoral.com/tag/violencia-narco/...,"Acribillaron a Benito Aguas Atlahua, diputado ...",violencia_narco,ellitoral
3,https://www.ellitoral.com/tag/violencia-narco/...,"""Esto nunca va a terminar"": volvieron a amenaz...",violencia_narco,ellitoral
4,https://www.ellitoral.com/tag/violencia-narco/...,Detuvieron en Buenos Aires a prófugo por el cr...,violencia_narco,ellitoral
...,...,...,...,...
482,https://www.ellitoral.com/tag/narcotrafico/suc...,"Detención de ""Cachiporri"": la ministra Bullric...",narcotrafico,ellitoral
483,https://www.ellitoral.com/tag/narcotrafico/suc...,"Video: así fue la detención de ""Cachiporri"", e...",narcotrafico,ellitoral
484,https://www.ellitoral.com/tag/narcotrafico/suc...,"Santa Fe: detuvieron a ""Cachiporri"" por tráfic...",narcotrafico,ellitoral
485,https://www.ellitoral.com/tag/narcotrafico/reg...,Intendentes del departamento Castellanos solic...,narcotrafico,ellitoral


In [9]:
df_litoral["Link"] = df_litoral["Link"].str.replace("tag/violencia-narco/", "", regex=False)
df_litoral["Link"] = df_litoral["Link"].str.replace("/tag/narcotrafico", "", regex=False)


In [10]:
df_litoral["Link"][0]

'https://www.ellitoral.com/internacionales/asesinaron-tiros-alcalde-personas-mexico-ahora-hoy-jesus-franco-larraga-tancanhuitz-san-luis-potosi_0_4idalwDbKK.html'

In [12]:
def content_and_date_extract(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content.decode('utf-8', errors='ignore'), 'html.parser')
        
        # Extract paragraphs
        paragraphs = soup.findAll('p')
        content_list = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
        
        # Extract datetime attribute
        datetime_element = soup.find(attrs={"datetime": True})
        date = datetime_element['datetime'] if datetime_element else None

        # Return a tuple of content and date
        return {"content": content_list if content_list else ["No content found"], "date": date}
    except requests.exceptions.RequestException as e:
        return {"content": [f"Request error: {e}"], "date": None}
    except Exception as e:
        return {"content": [f"Error: {e}"], "date": None}

# Apply the function to the 'link' column
df_extracted = df_litoral['Link'].apply(content_and_date_extract)

# Create separate columns for 'content' and 'date'
df_litoral['content'] = df_extracted.apply(lambda x: x['content'])
df_litoral['date'] = df_extracted.apply(lambda x: x['date'])


In [13]:
df_litoral

,Link,Title,Tag,Media,content,date
0,https://www.ellitoral.com/internacionales/ases...,Asesinaron a tiros a un alcalde y otras tres p...,violencia_narco,ellitoral,"[Se trata de Jesús Franco Larraga, un mandatar...",2024-12-16T08:14:51-03:00
1,https://www.ellitoral.com/sucesos/carcel-narco...,A la cárcel: narcos barriales deberán pasar un...,violencia_narco,ellitoral,"[Se trata de Manuel Antonio Concina, su padre ...",2024-12-15T20:17:59-03:00
2,https://www.ellitoral.com/internacionales/acri...,"Acribillaron a Benito Aguas Atlahua, diputado ...",violencia_narco,ellitoral,[El hombre del Partido Verde Ecologista fue at...,2024-12-10T08:27:12-03:00
3,https://www.ellitoral.com/sucesos/va-terminar-...,"""Esto nunca va a terminar"": volvieron a amenaz...",violencia_narco,ellitoral,[Un grupo de delincuentes publicó un video con...,2024-12-02T06:23:03-03:00
4,https://www.ellitoral.com/sucesos/detencion-vi...,Detuvieron en Buenos Aires a prófugo por el cr...,violencia_narco,ellitoral,[Fue detenido en un operativo realizado en la ...,2024-11-26T23:57:02-03:00
...,...,...,...,...,...,...
482,https://www.ellitoral.com/sucesos/cachiporri-d...,"Detención de ""Cachiporri"": la ministra Bullric...",narcotrafico,ellitoral,[La titular del Ministerio de Seguridad de la ...,2024-02-13T21:24:24-03:00
483,https://www.ellitoral.com/sucesos/cachiporri-d...,"Video: así fue la detención de ""Cachiporri"", e...",narcotrafico,ellitoral,"[Gabriel Alberto Nudel, líder de ""La Negrada"",...",2024-02-13T19:19:11-03:00
484,https://www.ellitoral.com/sucesos/operativo-al...,"Santa Fe: detuvieron a ""Cachiporri"" por tráfic...",narcotrafico,ellitoral,"[Está vinculado con el narco Vicente Pignata, ...",2024-02-13T17:17:55-03:00
485,https://www.ellitoral.com/regionales/encuentro...,Intendentes del departamento Castellanos solic...,narcotrafico,ellitoral,[Participaron funcionarios municipales de Sunc...,2024-02-10T11:18:47-03:00


In [24]:
df_litoral.to_excel("data/df_litoral_final.xlsx", index=False)

In [19]:
df_litoral['date'] = pd.to_datetime(df_litoral['date'], errors='coerce')

In [23]:
df_litoral['date'] = df_litoral["date"].dt.date